In [56]:
#dependencies
from pathlib import Path
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
from datetime import datetime


Main_csv_path = Path("Resources/18100205.csv")
Metadata_csv_path = Path("Resources/18100205_MetaData.csv")

Main_StatsCan_IndexDF = pd.read_csv(Main_csv_path, encoding="UTF-8-Sig")
Metadata_DF = pd.read_csv(Metadata_csv_path, encoding="UTF-8-Sig")

Main_StatsCan_IndexDF.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
61075,2023-05,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,units,0,v111955557,39.2,126.4,NaN,NaN,NaN,1
61076,2023-05,"Vancouver, British Columbia",2011S0503933,Land only,"Index, 201612=100",347,units,0,v111955558,39.3,121.0,E,NaN,NaN,1
61077,2023-05,"Victoria, British Columbia",2011S0503935,Total (house and land),"Index, 201612=100",347,units,0,v111955559,40.1,121.9,NaN,NaN,NaN,1
61078,2023-05,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,units,0,v111955560,40.2,127.9,NaN,NaN,NaN,1
61079,2023-05,"Victoria, British Columbia",2011S0503935,Land only,"Index, 201612=100",347,units,0,v111955561,40.3,110.6,E,NaN,NaN,1


In [4]:
Metadata_DF.head()

,Cube Title,Product Id,CANSIM Id,URL,Cube Notes,Archive Status,Frequency,Start Reference Period,End Reference Period,Total number of dimensions
"New housing price index, monthly",18100205,327-0056,https://www150.statcan.gc.ca/t1/tbl1/en/tv.act...,1;2;3;13;14,CURRENT - a cube available to the public and t...,Monthly,1981-01-01,2023-05-01,2.0,NaN
Dimension ID,Dimension name,Dimension Notes,Dimension Definitions,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geography,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,New housing price indexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dimension ID,Member Name,Classification Code,Member ID,Parent Member ID,Terminated,Member Notes,Member Definitions,NaN,NaN,NaN


In [5]:
# Select necessary columns for initial data cleaning processes.

reduced_main_df = Main_StatsCan_IndexDF[["REF_DATE","GEO","New housing price indexes","UOM","VALUE","STATUS","DECIMALS"]]		
#reduced_main_df.head()

reduced_main_df.columns.values[2] = "NewHousePrice_Index_Type" #Replaced column name of "New housing price indexes".

reduced_main_df.tail()

,REF_DATE,GEO,NewHousePrice_Index_Type,UOM,VALUE,STATUS,DECIMALS
61075,2023-05,"Vancouver, British Columbia",House only,"Index, 201612=100",126.4,NaN,1
61076,2023-05,"Vancouver, British Columbia",Land only,"Index, 201612=100",121.0,E,1
61077,2023-05,"Victoria, British Columbia",Total (house and land),"Index, 201612=100",121.9,NaN,1
61078,2023-05,"Victoria, British Columbia",House only,"Index, 201612=100",127.9,NaN,1
61079,2023-05,"Victoria, British Columbia",Land only,"Index, 201612=100",110.6,E,1


In [6]:
#import re
#print(re.split(r',',reduced_main_df["GEO"]))
Orig_Geo_series = pd.Series(reduced_main_df["GEO"])
#print(Orig_Geo_series)
Geo_Split = Orig_Geo_series.str.split(',')

Geo_CityRow = Geo_Split.str.get(0) #access the first element for eventual "column1"

Geo_CityRow

0                 Canada
1                 Canada
2                 Canada
3        Atlantic Region
4        Atlantic Region
              ...       
61075          Vancouver
61076          Vancouver
61077           Victoria
61078           Victoria
61079           Victoria
Name: GEO, Length: 61080, dtype: object

In [7]:
Geo_ProvRow = Geo_Split.str.get(1) #access the first element for eventual "column2"

Geo_ProvRow

0                      NaN
1                      NaN
2                      NaN
3                      NaN
4                      NaN
               ...        
61075     British Columbia
61076     British Columbia
61077     British Columbia
61078     British Columbia
61079     British Columbia
Name: GEO, Length: 61080, dtype: object

In [8]:
#Temp_DF = pd.concat(reduced_main_df,Geo_City )

Manual_Merge_Geo = pd.concat([Geo_CityRow,Geo_ProvRow], axis=1) #axis=1 is using "column 1" to merge.

Manual_Merge_Geo

#Rename Options presented via => https://www.geeksforgeeks.org/rename-specific-columns-in-pandas/
Manual_Merge_Geo.columns.values[0] = "City_Region"
Manual_Merge_Geo.columns.values[1] = "Province"

Manual_Merge_Geo

#BEST PRACTICE MIGHT BE TO RENAME RIGHT AWAY "HERE" ... to save coding time later.

,City_Region,Province
0,Canada,NaN
1,Canada,NaN
2,Canada,NaN
3,Atlantic Region,NaN
4,Atlantic Region,NaN
...,...,...
61075,Vancouver,British Columbia
61076,Vancouver,British Columbia
61077,Victoria,British Columbia
61078,Victoria,British Columbia


In [9]:
Orig_Year_series = pd.Series(reduced_main_df["REF_DATE"])
Year_Split = Orig_Year_series.str.split('-')

Row_Year = Year_Split.str.get(0) #access the first element for eventual "column1"

Row_Year

0        1981
1        1981
2        1981
3        1981
4        1981
         ... 
61075    2023
61076    2023
61077    2023
61078    2023
61079    2023
Name: REF_DATE, Length: 61080, dtype: object

In [10]:
Row_Month = Year_Split.str.get(1) #access the first element for eventual "column1"

Row_Month

0        01
1        01
2        01
3        01
4        01
         ..
61075    05
61076    05
61077    05
61078    05
61079    05
Name: REF_DATE, Length: 61080, dtype: object

In [11]:
Manual_Merge_date = pd.concat([Row_Year,Row_Month], axis=1) #axis=1 is using "column 1" to merge.

Manual_Merge_date

#Rename Options presented via => https://www.geeksforgeeks.org/rename-specific-columns-in-pandas/
Manual_Merge_date.columns.values[0] = "Year"
Manual_Merge_date.columns.values[1] = "Month"

Manual_Merge_date
#BEST PRACTICE MIGHT BE TO RENAME RIGHT AWAY "HERE" ... to save coding time later.
#KEY REMINDER ... if we need the month to be transformed into "1" intstead of "01" (IE: INT from TEXT/VARCHAR) ... here is probably the best place as well.

,Year,Month
0,1981,01
1,1981,01
2,1981,01
3,1981,01
4,1981,01
...,...,...
61075,2023,05
61076,2023,05
61077,2023,05
61078,2023,05


In [12]:
#via => https://appdividend.com/2020/06/02/pandas-concat-method-example-in-python/
#JOIN Split GEO Columns to Original DF

GeoUpdated_DF = pd.concat([reduced_main_df,Manual_Merge_Geo],axis=1,join='outer') # ,join_axes=None,ignore_index=False)

GeoUpdated_DF

#JOIN Split YEar+Month Columns to GeoUpdated_DF

DateUpdated_DF = pd.concat([GeoUpdated_DF,Manual_Merge_date],axis=1,join='outer')

DateUpdated_DF


,REF_DATE,GEO,NewHousePrice_Index_Type,UOM,VALUE,STATUS,DECIMALS,City_Region,Province,Year,Month
0,1981-01,Canada,Total (house and land),"Index, 201612=100",38.2,NaN,1,Canada,NaN,1981,01
1,1981-01,Canada,House only,"Index, 201612=100",36.1,NaN,1,Canada,NaN,1981,01
2,1981-01,Canada,Land only,"Index, 201612=100",40.6,E,1,Canada,NaN,1981,01
3,1981-01,Atlantic Region,Total (house and land),"Index, 201612=100",NaN,..,1,Atlantic Region,NaN,1981,01
4,1981-01,Atlantic Region,House only,"Index, 201612=100",NaN,..,1,Atlantic Region,NaN,1981,01
...,...,...,...,...,...,...,...,...,...,...,...
61075,2023-05,"Vancouver, British Columbia",House only,"Index, 201612=100",126.4,NaN,1,Vancouver,British Columbia,2023,05
61076,2023-05,"Vancouver, British Columbia",Land only,"Index, 201612=100",121.0,E,1,Vancouver,British Columbia,2023,05
61077,2023-05,"Victoria, British Columbia",Total (house and land),"Index, 201612=100",121.9,NaN,1,Victoria,British Columbia,2023,05
61078,2023-05,"Victoria, British Columbia",House only,"Index, 201612=100",127.9,NaN,1,Victoria,British Columbia,2023,05


In [13]:
#reduced_main_df = Main_StatsCan_IndexDF[["REF_DATE","GEO","New housing price indexes","UOM","VALUE","STATUS","DECIMALS"]]

Analysis_DF = DateUpdated_DF[["Year","Month","City_Region","Province","NewHousePrice_Index_Type","UOM","VALUE","STATUS","DECIMALS"]]

Analysis_DF

,Year,Month,City_Region,Province,NewHousePrice_Index_Type,UOM,VALUE,STATUS,DECIMALS
0,1981,01,Canada,NaN,Total (house and land),"Index, 201612=100",38.2,NaN,1
1,1981,01,Canada,NaN,House only,"Index, 201612=100",36.1,NaN,1
2,1981,01,Canada,NaN,Land only,"Index, 201612=100",40.6,E,1
3,1981,01,Atlantic Region,NaN,Total (house and land),"Index, 201612=100",NaN,..,1
4,1981,01,Atlantic Region,NaN,House only,"Index, 201612=100",NaN,..,1
...,...,...,...,...,...,...,...,...,...
61075,2023,05,Vancouver,British Columbia,House only,"Index, 201612=100",126.4,NaN,1
61076,2023,05,Vancouver,British Columbia,Land only,"Index, 201612=100",121.0,E,1
61077,2023,05,Victoria,British Columbia,Total (house and land),"Index, 201612=100",121.9,NaN,1
61078,2023,05,Victoria,British Columbia,House only,"Index, 201612=100",127.9,NaN,1


In [14]:
#Remove "junk data" columns based on StatsCan Legend.

StatusClean_Analysis_DF = Analysis_DF.query("STATUS not in ['E', 'x', '..']") # QUERY fuction delivers to python "memory" a "true/false" outcome.
TotalPriceIndex_Analysis_DF = StatusClean_Analysis_DF.query("NewHousePrice_Index_Type not in ['House only','']")
#StatusClean_Analysis_DF

TotalPriceIndex_Analysis_DF

,Year,Month,City_Region,Province,NewHousePrice_Index_Type,UOM,VALUE,STATUS,DECIMALS
0,1981,01,Canada,NaN,Total (house and land),"Index, 201612=100",38.2,NaN,1
9,1981,01,St. John's,Newfoundland and Labrador,Total (house and land),"Index, 201612=100",36.1,NaN,1
27,1981,01,Saint John,Fredericton,Total (house and land),"Index, 201612=100",55.9,NaN,1
33,1981,01,Québec,Quebec,Total (house and land),"Index, 201612=100",30.1,NaN,1
42,1981,01,Montréal,Quebec,Total (house and land),"Index, 201612=100",28.5,NaN,1
...,...,...,...,...,...,...,...,...,...
61065,2023,05,Edmonton,Alberta,Total (house and land),"Index, 201612=100",108.2,NaN,1
61068,2023,05,British Columbia,NaN,Total (house and land),"Index, 201612=100",127.0,NaN,1
61071,2023,05,Kelowna,British Columbia,Total (house and land),"Index, 201612=100",123.8,NaN,1
61074,2023,05,Vancouver,British Columbia,Total (house and land),"Index, 201612=100",127.8,NaN,1


In [15]:
desired_cities = ['Toronto', 'Vancouver', 'Montréal']
filtered_city_data = {}

for city in desired_cities:
    city_filtered_df = TotalPriceIndex_Analysis_DF[TotalPriceIndex_Analysis_DF['City_Region'] == city]
    filtered_city_data[city] = city_filtered_df

# Print the data for all cities
for city, city_df in filtered_city_data.items():
    print(f"Data for {city}:")
    print(city_df)
    print("\n")

Data for Toronto:
       Year Month City_Region  Province NewHousePrice_Index_Type  \
57     1981    01     Toronto   Ontario   Total (house and land)   
177    1981    02     Toronto   Ontario   Total (house and land)   
297    1981    03     Toronto   Ontario   Total (house and land)   
417    1981    04     Toronto   Ontario   Total (house and land)   
537    1981    05     Toronto   Ontario   Total (house and land)   
...     ...   ...         ...       ...                      ...   
60537  2023    01     Toronto   Ontario   Total (house and land)   
60657  2023    02     Toronto   Ontario   Total (house and land)   
60777  2023    03     Toronto   Ontario   Total (house and land)   
60897  2023    04     Toronto   Ontario   Total (house and land)   
61017  2023    05     Toronto   Ontario   Total (house and land)   

                     UOM  VALUE STATUS  DECIMALS  
57     Index, 201612=100   28.0    NaN         1  
177    Index, 201612=100   28.5    NaN         1  
297    Index

In [16]:
#Extract sample data to validate that the STATUS not in funciton above was successful.
Output_Folder = 'Output/'

utput_Folder = 'Output/'
Output_file = Output_Folder + 'city_filtered_df.csv'

# Concatenate DataFrames for all cities
combined_data = pd.concat(filtered_city_data.values(), ignore_index=True)

# Save the combined data to a single CSV file
combined_data.to_csv(Output_file, index=False, encoding='utf-8-sig') # ref for variation in export encoding https://stackoverflow.com/questions/57152985/what-is-the-difference-between-utf-8-and-utf-8-sig


In [49]:
Housing_Prices_by_year = Path("Resources/Housing_Prices_by_year.csv")
Housing_Prices_by_year_df = pd.read_csv(Housing_Prices_by_year, encoding="UTF-8-Sig")

Housing_Prices_by_year_df

,month/Year,Vancouver,Toronto,Montreal
0,12-Jan,"$660,600","$420,950","284,000"
1,12-Feb,"$670,900","$456,537",NaN
2,12-Mar,"$679,000","$450,750",NaN
3,12-Apr,"$683,800","$464,075",NaN
4,12-May,"$625,100","$472,839",NaN
...,...,...,...,...
127,22-Aug,"$1,180,500","$939,625","$523,700"
128,22-Sep,"$1,155,300","$957,500","$511,400"
129,22-Oct,"$1,148,900","$964,125","$504,800"
130,22-Nov,"$1,131,600","$964,125","$501,000"


In [62]:
month_Year = '2023-12'
date_format = '%Y-%m'
date_object = datetime.strptime(month_Year, date_format)

# Create a new datetime object with only the year and month
year_month_object = datetime(date_object.year, date_object.month, 1)

print(year_month_object)

2023-12-01 00:00:00


C:\Users\64724\AppData\Local\Temp\ipykernel_29932\912383338.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  Housing_Prices_by_year_df['month/Year'] = pd.datetime.date(Housing_Prices_by_year_df['month/Year'])


TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'Series' object

In [43]:
# Housing_Prices_by_year_df['Text_Date'] = pd.to_datetime('2012-' + Housing_Prices_by_year_df['month/Year'], format='%Y-%d-%b').dt.strftime('%Y-%m-%d')

# Housing_Prices_by_year_df.()

,month/Year,Vancouver,Toronto,Montreal,Text_Date
127,22-Aug,"$1,180,500","$939,625","$523,700",2012-08-22
128,22-Sep,"$1,155,300","$957,500","$511,400",2012-09-22
129,22-Oct,"$1,148,900","$964,125","$504,800",2012-10-22
130,22-Nov,"$1,131,600","$964,125","$501,000",2012-11-22
131,22-Dec,"$1,114,300","$939,125","$516,400",2012-12-22


In [31]:
HP_by_year_df = (Housing_Prices_by_year_df["month/Year"])
monthYear_Split = (Orig_Year_series.str.split('-'))

monthYear_Split

0        [1981, 01]
1        [1981, 01]
2        [1981, 01]
3        [1981, 01]
4        [1981, 01]
            ...    
61075    [2023, 05]
61076    [2023, 05]
61077    [2023, 05]
61078    [2023, 05]
61079    [2023, 05]
Name: REF_DATE, Length: 61080, dtype: object

NameError: name 'HPRow_Year' is not defined

In [15]:
#2 Rename Options presented via => https://www.geeksforgeeks.org/rename-specific-columns-in-pandas/
#OPTION1
# renaming the columns
#df.rename({"Name": "Student Name", 
#           "Marks": "Marks Obtained", 
#           "Roll No": "Roll Number"}, 
#          axis = "columns", inplace = True)

#OPTION2
# using the same modified dataframe 
# df from Renaming Multiple Columns
# Renaming the third column
#df.columns.values[2] = "Roll Number"

#FIRST Need to transpose the DF col names to a list... using via => https://stackoverflow.com/questions/27129152/pandas-column-names-to-list#:~:text=Simple%20and%20easy%20way%3A%20df-dataframe%20variable%20name%20df.columns.to_list,give%20the%20list%20of%20the%20all%20columns%20name.

#ColName_DF = DateUpdated_DF.columns.get_value()



In [16]:
#next step1 is to merge the 2 VARs to the reduced_main_df ... and then remove the original "GEO" column.
#next step2 is to use pinned screen to "filter" our results to only what we want from the "STATUS" column.
#STATUS column filters are as follows.. REMOVE the "..", "E" and "x".... (keep blanks - "NaN" values) ... 
#NEW HOUSING PRICE INDEX .... column to be "Total (house and land)" ... SO REMOVE "House only" & "Land only"

#ALSO ... complete the same for "Geo_City" .. etc ... for "year+month" ... from the "REF_DATE" column ...
#GEO column ... we will "focus on keeping" ... which will be "Toronto, Vancouver and Montreal" ... 

#RENAME the columns to stuff that "makes common sense" ... for our presenting needs (which is dependent on the base prices).

In [17]:
#SET UP the basepoints for the calculation with historical prices ...
#Create VARs to enable them to run against the cleaned data set ...

#Determine how to complete the analysis ... and fit into how they can be placed into slides.

#Confirm the types of "plots" ... we're confident in to speak to the story of the data analysis we're able to produce.
#Remember to comment that our $ costs are "approximations" ... due to needing the baseprice assumptions.

In [18]:
#Slides Notes:
#Maria is testing
